# Review analysis of Blue Bottle (1) - 데이터 수집
20.05.07
* 데이터 분석을 위한 크롤러 만들기
    * 카카오맵에서 블루보틀 각 지점별 리뷰 가져오기

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-데이터-크롤링" data-toc-modified-id="1.-데이터-크롤링-1">1. 데이터 크롤링</a></span></li><li><span><a href="#2.-데이터-확인" data-toc-modified-id="2.-데이터-확인-2">2. 데이터 확인</a></span></li><li><span><a href="#3.-중복값-확인해보기" data-toc-modified-id="3.-중복값-확인해보기-3">3. 중복값 확인해보기</a></span></li><li><span><a href="#4.-데이터-저장" data-toc-modified-id="4.-데이터-저장-4">4. 데이터 저장</a></span></li></ul></div>

## 1. 데이터 크롤링
* selenium, BeautifulSoup 사용

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid')
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup
from selenium import webdriver
import time

In [2]:
plt.rc('font', family='NanumGothic')
plt.rc('font', size=13)

In [124]:
driver = webdriver.Chrome('./chromedriver/chromedriver.exe')
driver.implicitly_wait(5)
# 지도에서 키워드 검색
keyword = '블루보틀'

driver.get('https://map.kakao.com/')
driver.implicitly_wait(3)
time.sleep(2)
try:
    driver.find_element_by_xpath('/html/body/div[10]/div/div/div/p').click()
except:
    pass
driver.find_element_by_id('search.keyword.query').send_keys(keyword)
time.sleep(1)
driver.find_element_by_id('search.keyword.submit').click()
driver.implicitly_wait(3)
time.sleep(3)


# driver.find_element_by_xpath(
#     '//*[@id="info.search.place.list"]/li[2]/div[4]/span[1]/a').click()

data = {'cafe': [], 'review': [], 'score': []}
map_pagenum = 0

try:
    map_pagenum += 1
    # 페이지 이동 버튼의 id넘버가 5의 나머지로 되어있다. (1페이지 = p1, 6페이지 = p1, 7페이지 = p2 ...)
    map_page = map_pagenum % 5

    # 각 카페들의 리뷰 조회
    try:
        for cafenum in range(15):
            pagenum = 0
            # 리뷰 크롤링 시작
            try:
                driver.find_element_by_xpath(
                    '//*[@id="info.search.place.list"]/li[{}]/div[4]/span[1]/a'.format(cafenum+1)).click()
                driver.implicitly_wait(3)
                time.sleep(3)

                window_name = driver.window_handles[1]
                driver.switch_to.window(window_name=window_name)
                while True:
                    try:
                        pagenum += 1    # 페이지넘버 카운트
                        soup = BeautifulSoup(driver.page_source, 'lxml')
                        # 리뷰가 몇 개 있는지 카운트
                        count = len(soup.find_all('p', 'txt_comment'))
                        for i in range(count):
                            data['cafe'].append(soup.find('title').text[:-7])
                            data['review'].append(
                                soup.find_all('p', 'txt_comment')[i].text)
                            data['score'].append(soup.find_all(
                                'em', 'num_rate')[i+2].text)
                            time.sleep(1)

                        if pagenum % 5 != 0:
                            if pagenum <= 5:
                                driver.find_element_by_xpath(
                                    '//*[@id="mArticle"]/div[4]/div[4]/div/a[{}]'.format(pagenum)).click()
                            else:
                                driver.find_element_by_xpath(
                                    '//*[@id="mArticle"]/div[4]/div[4]/div/a[{}]'.format((pagenum % 5)+1)).click()
                        else:
                            driver.find_element_by_class_name(
                                'btn_next').click()

                        driver.implicitly_wait(3)
                        time.sleep(2)

                    except:
                        print('끝!')
                        break
                driver.close()
                first_tab = driver.window_handles[0]
                driver.switch_to.window(window_name=first_tab)
                time.sleep(2)
            # 페이지가 5페이지씩 나뉘어져있는데(1-5, 6-10..), 마지막 페이지(5의 배수 페이지)에 도착했을 경우, 다음 버튼 클릭
            except:
                pass
    except:
        print('{}페이지의 카페 수가 15개 미만'.format(map_pagenum))
    if map_page == 0:
        driver.find_element_by_id('info.search.page.next').click()
    else:
        driver.find_element_by_id(
            'info.search.page.no{}'.format(map_page+1)).click()
    time.sleep(1)
except:
    print('마지막 페이지: {}'.format(map_pagenum))

끝!
끝!
끝!
끝!
끝!
끝!
페이지 초과: 1


## 2. 데이터 확인

In [125]:
len(data['cafe']), len(data['review']), len(data['score'])

(204, 204, 204)

In [138]:
data['review'][:10]

['불친절해요. 배가 불렀는듯더보기',
 '아메리카노도 맛있고 놀라플로트도 맛있어요!! 사람만 안많으면 자주오고싶어요더보기',
 '좋습니다더보기',
 '쏘쏘. 그냥 테이크아웃하기 좋음더보기',
 '편히 머물 수 있는 공간은 아닌. 커피는 우유들어간건 맛있어요.. 디저트는 메종엠오더보기',
 '이슈가 많길래 호기심에 가봤는데 커피가 생각 외로 맛도 없고;; 또 불친절해서 놀랐어요. 그리고 가격 너무 비싸서 좀 황당할정도... ㅎ위치도 별로고... 양도 적고... 더보기',
 '에티오피아 리무 두로미나 워시드 드립 생두 단가를 고려할때 가격은 적당. 추출자체도 준수하게 잘됨.  가격대비 퀄리티는 브랜드값 떼고도 괜찮다고 생각한다. 적어도 내가 본 직원은 친절했음. 다만 여길 밖까지 줄서서 마실 정도는 아니다.  그냥 괜찮은 카페일뿐.  내가 갔을 땐 줄이 없었고. 실내 좌석수가 많지는 않다. 단층이고 테이크아웃 위주로 고려하고 만...더보기',
 '스벅은 별이라도 준다. 더보기',
 '최고예요더보기',
 '갈때마다 미소로 반겨주는 직원들에 맛있는 커피까지.. 역시 기대했던 만큼 만족스러웠어요. 또 가고 싶습니다. 더보기']

In [140]:
data['review'][-10:]

['파란병이던데요.... 빨대도..더보기',
 '간판찍고옴더보기',
 '한 여름 대낮에 줄을 서서 커피를 마신다는 것이 얼마나 열정이 있어야 하는 일인지... 오후 5시쯤 되니 오히려 줄이 줄었다.   층마다 바리스타가 있고, 굉장히 기능에 집중 한 공간.  그리고, 점점 높아져가는 커피가격. 더보기',
 '인테리어가 멋졌어요더보기',
 '성수점보다 훌륭한 건축과 인테리어 마감. 아이덴티티. 한옥과의 조화.. 개방감.. 더보기',
 '사람들이 다 한방향으로 달려가길래 나는 무슨 연예인이 온 줄 알았다 이 땡볕에도 간판과 사진을 찍기 위해 그리고 가게 안으로 들어가기 위해 줄을 서있다 대한민국사람들 정말 위대하다더보기',
 '나가지도 않았는데 잔부터 치워버리는건 무슨 생각이지...? 옆에 짐도 버젓히 놔뒀는데..? 다마셨으면 빨리 나가란 건가?더보기',
 '최고예요더보기',
 '내 입맛은 뉴요커가 아닌가봐요더보기',
 '드디어 카카오 맵에 장소 등록됐네요.. 삼청 한옥점이랑은 다릅니다.더보기']

* 긁어온 리뷰 마지막에 '더보기' 라는 텍스트를 같이 가져왔다.
* 해당 텍스트 삭제하자.

In [151]:
data['review'][0]

'불친절해요. 배가 불렀는듯'

In [149]:
data['review'] = [s[:-3] for s in data['review']]
data['review'][0]

'불친절해요. 배가 불렀는듯'

* 텍스트 삭제 성공!

## 3. 중복값 확인해보기
* 웹 크롤링 중에 time.sleep을 적절하게 사용하지 않으면, 값이 밀려서 저장되는 경우가 있다.
* 위와 같은 오류가 생기면 중복값이 자주 생긴다.
    * 여기서 말하는 중복값은 실제 값은 [A, B, C]인데, [A, A, A]라고 내용을 저장해버리는 것.
    * 실제 값도 [A, A, A]인데, [A, A, A]라고 저장되는 것은 에러가 아니니까 위의 경우만 확인해주면 됨.
* 이런 이슈가 없는지 확인해보자.

In [156]:
df = pd.DataFrame(data)
df.head()

,cafe,review,score
0,블루보틀 역삼점,불친절해요. 배가 불렀는듯,2점
1,블루보틀 역삼점,아메리카노도 맛있고 놀라플로트도 맛있어요!! 사람만 안많으면 자주오고싶어요,5점
2,블루보틀 역삼점,좋습니다,5점
3,블루보틀 역삼점,쏘쏘. 그냥 테이크아웃하기 좋음,3점
4,블루보틀 역삼점,편히 머물 수 있는 공간은 아닌. 커피는 우유들어간건 맛있어요.. 디저트는 메종엠오,2점


In [155]:
df[df['review'].duplicated()]

,cafe,review,score
15,블루보틀 역삼점,조금 아쉬워요,2점
26,블루보틀 압구정점,별로예요,1점
54,블루보틀 성수점,보통이에요,3점
63,블루보틀 성수점,보통이에요,3점
65,블루보틀 성수점,보통이에요,3점
66,블루보틀 성수점,보통이에요,3점
78,블루보틀 성수점,보통이에요,3점
84,블루보틀 성수점,보통이에요,3점
100,블루보틀 성수점,최고예요,5점
132,블루보틀 성수점,최고예요,5점


* 중복값이 있긴 한데, 일반적으로 많이 남기는 리뷰이긴 하다.
* 실제로 리뷰들을 확인해보니, 에러로 인한 중복이 아니었으므로 조정해줄 필요가 없다.

## 4. 데이터 저장
* 이제 데이터를 저장하고, 분석해보자.

In [157]:
df.to_csv('source/Bluebottle_review/data.csv')